# YAHOO電影爬蟲練習
## 這個作業主要是練習爬取YAHOO電影的資訊，主要會用到基本的GET method，以及解析伺服器回傳的html，從中擷取我們想要的資料。

### 這個範例示範如何爬取上映中的電影資訊。先訪問YAHOO電影首頁：https://movies.yahoo.com.tw ，點選「上映中」的分頁，先確認上映中的電影總共有幾筆，然後逐頁爬取。

In [3]:
import requests
import re
from bs4 import BeautifulSoup

In [2]:
# 先觀察一下目前上映中的電影數量
url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("div", attrs={'class':'release_box'})  # 尋找正在上映中的全部電影筆數，其所在的tag
print("正在上映中總共: ", html.p.string)

正在上映中總共:  共80筆，目前顯示1~10筆


### 爬取所有上映中的電影資料：電影名稱、期待度、滿意度、簡介等等。

In [4]:
# 接下來要爬取每一頁的電影介紹
# 先點選網頁最底部的下一頁，觀察其網址變化。然後我們把頁數當作變數傳入，用迴圈爬取。
max_page = 5
for page_number in range(1, max_page+1, 1):
    url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
    payload = {'page':str(page_number)}
    resp = requests.get(url, params=payload)
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text, 'lxml')

    # 把電影介紹擷取出來
    movie_list = soup.find("ul", attrs={"class":"release_list"}).find_all("li")
    for p in movie_list:
        print("----------------------------------------------------------------------")
        # 電影名稱
        movie_name = p.find("div", attrs={"class":"release_movie_name"})
        print("電影名稱：", movie_name.a.string)
        
        # 定位電影評價資訊欄位
        level_box = movie_name.find("dl", attrs={"class":"levelbox"})
        
        # 期待度
        expectation = level_box.find("div", attrs={"class":"leveltext"})
        print("期待度：", expectation.span.string)
        
        # 滿意度
        #satisfaction = level_box.find("div", attrs={"class":"leveltext starwithnum"})       
        satisfaction = level_box.find_all("div", attrs={"class":"leveltext"})       
        print("滿意度：", satisfaction[1].span["data-num"])  # 滿意度可以從"data-num"這個屬性擷取
        #print("滿意度：", satisfaction.span.string)  # 滿意度可以從"data-num"這個屬性擷取
        
        # 簡介
        movie_info = p.find("div", attrs={"class":"release_text"})
        print(movie_info.span.string)


----------------------------------------------------------------------
電影名稱： 
                  依然相信
期待度： 85%
滿意度： 4.2

                  ★歌手傑洛米坎普真實人生改編
★《河谷鎮》人氣小鮮肉KJ阿帕深情演出
★冠軍單曲〈我依然相信〉（I Still Believe）背後的真實故事
★《夢想心樂章》製片團隊浪漫打造
 
改編自歌手傑洛米坎普（Jeremy Camp）真實人生，兒時因為在學校廚房被挖角唱歌天賦，就此展開了音樂旅程，至今出過11張專輯，其中四張經由美國唱片協會認證為金唱片，得過五座金鴿獎，以及四座ASCAP年度最佳作詞人獎。就在人生順遂之際，未婚妻卻罹患絕症，隨著時間倒數，兩人該如何一同面對困難，迎接所剩不多的人生？
                
----------------------------------------------------------------------
電影名稱： 
                  我出去醫下
期待度： 74%
滿意度： 3.8

                  ★ 笑彈出擊！勇奪法國新片票房冠軍
★ 坎城影帝、最佳劇本得主米歇爾布朗爆笑主演
★ 法國當紅YouTuber哈基姆傑米利大銀幕初登場
★ 美食外送員誤打誤「撞」成醫生？隔空指導看診有夠瞎！
★ 凱薩獎最佳首部電影、最佳紀錄片得主最新製片力作
★ 《劫局》《推銷員之戀》攝影師X《今夜，我們無罪》《烈火焚身》作曲家傾力製作
 
前無國界醫生賽吉是唯一一位會在聖誕夜出診的醫生，他跟美食外送員馬利克發生了車禍。雖然被撞的是馬利克，但卻引發了賽吉的坐骨神經痛，情急之下，馬利克只好代替賽吉登門看診。於是他戴上AirPods，透過賽吉的隔空電話指導，搖身一變成為超瞎醫生！
                
----------------------------------------------------------------------
電影名稱： 
                  最後晚安曲
期待度： 84%
滿意度： 3.8

                  ★集結奧斯卡得主與提名卡司，包含艾

----------------------------------------------------------------------
電影名稱： 
                  電影版 地獄少女
期待度： 91%
滿意度： 3.6

                  ★ 超人氣同名動畫首部真人電影版
★ 《詛咒》《貞子vs伽椰子》導演白石晃士最新力作 展現獨特暴力恐怖風格
★ 《惡之華》玉城蒂娜 × 《天氣之子》森七菜 新生代人氣演員竭力熱演
 
《地獄少女》以一個都市傳說為出發點，講述一個只有在午夜零時才可以連線的網站「地獄通信」，只要在網站上打上怨恨之人的名字，「地獄少女」閻魔愛便會替你復仇，將被寫下名字的人流放到「地獄」。不過詛咒者的代價，便是自己的靈魂也會在死後流放地獄，永世不得超生……
 
                
----------------------------------------------------------------------
電影名稱： 
                  血衛
期待度： 86%
滿意度： 3.5

                  ★《玩命關頭系列》製作團隊強力打造　場面刺激再創視覺震撼饗宴
★馮迪索化身超級英雄　展現超狂拳拳到肉打鬥場面
 
取材自暢銷漫畫，士兵雷葛瑞森 (馮迪索 飾) 在一次行動中陣亡，死後被帶至RST公司進行試驗，並成功以超人類「血衛」重生。他的血液中存在著㇐批奈米技術大軍，擁有一股不可阻擋的強大威力—不但能帶來比平常更猛的力量，還能夠及時痊癒。但除了他的身體，RST也同時抹去他舊有的記憶，並控制他的思想。雷無法分辨什麼是真實還是捏造的假象，而他的使命，就是要找出真相！
 
                
----------------------------------------------------------------------
電影名稱： 
                  我，最親愛的
期待度： 68%
滿意度： 4.0

                  ★ 「當《愛在暹邏》遇上《東京物語》」 ★
★「讓人聯想起《以你的名字呼喚我》的美好！」 ★
★【釜山影展】、【聖地牙哥亞洲電影節】、【夏威夷影展】全球影展競相邀約 
 
越南韓國

----------------------------------------------------------------------
電影名稱： 
                  絕命大平台
期待度： 79%
滿意度： 3.6

                  ★ 挖掘人性直逼《奪魂鋸》，禁閉設定媲美《異次元殺陣》
★ 反烏托邦宛如《摩天樓》，為了自由，你能堅持多久？
★ 《羊男的迷宮》伊萬馬薩格╳《我的母親》安東妮亞聖胡安領銜主演
★ 《傑克蓋的房子》拉斯馮提爾西班牙製片公司Zentropa出品，柏林影展泰迪熊獎製片最新力作
★ 《浩劫奇蹟》《靈異孤兒院》哥雅獎最佳剪輯得主操刀新作
★ 2019多倫多國際影展「午夜瘋狂」單元觀眾票選獎
★ 2019西班牙錫切斯奇幻影展最佳影片、最佳特效、最佳新導演、觀眾票選最佳影片
★ 2019德州奧斯汀奇幻影展
★ 2020哥雅獎最佳新導演、最佳原創劇本、最佳特效提名
★ 2020高第獎最佳劇本、最佳視效、最佳非加泰隆尼亞語電影提名
★ 每個人都要吃，每個人都想活，每個人都想逃脫！
 
當葛蘭漸漸從昏迷中醒來時，發現自己身處在一座宛如監獄般建築的第48層，這座建築從地表往地底延伸，中間由巨大的方形洞貫穿，一個大平台會將上層住戶吃剩的食物一層層依序送下來。與他同一層的「獄友」崔馬加西知道關於這個神秘處所的規則：每層有兩個人，總人數不詳。如果有辦法往上就能生存，但如果想太多就會再次沉淪。如果在幾乎沒有食物送達的底層，除了自身的勇氣，誰都不能相信……
                
----------------------------------------------------------------------
電影名稱： 
                  電影音效傳奇：好萊塢之聲
期待度： 80%
滿意度： 3.3

                  ★繼《電影配樂傳奇》之後，又一部電影幕後聲匠職人精神紀錄片。
★奧斯卡歷屆最佳音效設計、音效剪輯得主齊聚暢談經典電影音效錄製秘辛
★李安、史蒂芬史匹柏、喬治盧卡斯、大衛林區等導演闡述各自音效美學理念。
★好萊塢八大影業再度聯合授權數十部經典電影音效、擬音片段。
 
「我始終認為聲音比影像更能引領觀眾融入故事」－名導 史蒂芬史匹柏
 
談起電影中的「聲音